In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

load_from_wiki = False

In [2]:
# https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Germany
if load_from_wiki:

    page = requests.get("https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Germany")

    soup = BeautifulSoup(page.content, 'html.parser')
    # print(soup.prettify())

    Bavaria_tables = soup.find_all("table", class_='wikitable')
    
    df_Bavaria = pd.read_html(str(Bavaria_tables), header=0, skiprows=1)[0]
    df_Bavaria.drop(index=[0, df_Bavaria.shape[0] - 1], inplace=True)
    
    df_Bavaria.to_csv("./csv_input/df_Bavaria_20200415.csv")
else:
    df_Bavaria = pd.read_csv("./csv_input/df_Bavaria_20200415.csv")

In [3]:
# print(df_Bavaria.head())

df_ncase = df_Bavaria[['Date', 'Bavaria']]
acc_death = \
df_ncase['Bavaria'].apply(lambda x: str(x).split('(')[1].split(')')[0] if len(str(x).split('(')) > 1 else 0).apply(lambda x: int(x)).to_numpy()

df_ncase['Bavaria'] = df_ncase['Bavaria'].apply(lambda x: str(x).split('(')[0]).apply(lambda x: int(x))

df_ncase.to_csv("./csv_input/df_Bavaria_cleaned_20200415.csv", index=False)
# df_ncase.head(100)

<ipython-input-3-fa90f333bbed>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ncase['Bavaria'] = df_ncase['Bavaria'].apply(lambda x: str(x).split('(')[0]).apply(lambda x: int(x))


In [4]:
acc_cases = df_ncase['Bavaria'].to_numpy()

new_cases = np.append(acc_cases[0], np.diff(acc_cases))
# print(new_cases.sum())
# print(acc_death)

In [24]:
# https://www.citypopulation.de/en/germany/admin/09__bayern/

Bavaria_pop = np.array([1200904, 1217533, 1638957, 1717926, 1701205, 2108710, 1536697, 1170153, 784636])
pop_ratio = Bavaria_pop/Bavaria_pop.sum()
# print(Bavaria_pop.sum())

13076721


In [22]:
# print(pop_ratio)

[0.09183525 0.0931069  0.12533394 0.13137284 0.13009416 0.16125679
 0.11751394 0.08948367 0.0600025 ]


In [ ]:
# According to Wikipedia, there are 28000 ICU beds as of 16-Apr-2020
# Using 83020000 as the total population of Germany and the ratio between 
# German total population:Bavaria total population, can get ICU = 13076721/83020000*28000 = 4410
t_icu = 4410 # Initial number of ICU beds
t_vent = t_icu*icu_with_vent_rate